## Load MOSAIKS features

In [ ]:
mosaiks_features_gdf = load_mosaiks_data("INDIA_SHRUG_Mosaiks_features.csv")
# mosaiks_features_gdf.head()

In [ ]:
mosaiks_features_gdf.shape

(94122, 4011)

## Load preprocessed SHRUG keys with shapes

In [ ]:
shrug_key_geoms = load_gdf("01_preprocessed/SHRUG/shrug_pc11r_key_with_shapes", "shrug_pc11r_key_with_shapes.shp")
shrug_key_geoms = lengthen_shapefile_ID_names(shrug_key_geoms)
# shrug_key_geoms.head()

## Use SHRUG shapes to add `shrid`s to MOSAIKS features

In [ ]:
mosaiks_features_gdf = add_shrid_to_mosaiks(mosaiks_features_gdf, shrug_key_geoms)

## Import SHRUG SECC (target)

In [ ]:
file_path = Path.cwd().parent / "data" / "00_raw" / "SHRUG" / "shrug-v1.5.samosa-secc-csv" / "shrug_secc.csv"
shrug_secc = pd.read_csv(file_path)
# shrug_secc.head()

In [ ]:
shrug_secc.shape

(565991, 11)

## Match target to features using `shrid`s

In [ ]:
df = merge_mosaiks_and_secc(mosaiks_features_gdf, shrug_secc)
# df.head()

In [ ]:
df.shape

(87984, 4021)